In [9]:
!pip install opencv-python transformers torch pillow


In [8]:
import cv2
import numpy as np
from transformers import pipeline
from PIL import Image

# Load depth model
pipe = pipeline(
    task="depth-estimation",
    model="depth-anything/Depth-Anything-V2-Large-hf",
    device=0
)

# Load video
cap = cv2.VideoCapture("/content/test.mp4")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("depth_output.mp4", fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR (OpenCV) → RGB (PIL)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(rgb)

    # Depth prediction
    depth = pipe(pil_image)["depth"]
    depth = np.array(depth)

    # Normalize depth map to 0-255
    depth = (depth - depth.min()) / (depth.max() - depth.min())
    depth = (depth * 255).astype(np.uint8)

    # Convert to 3-channel image
    depth_colored = cv2.cvtColor(depth, cv2.COLOR_GRAY2BGR)

    out.write(depth_colored)

cap.release()
out.release()

UnidentifiedImageError: cannot identify image file '/content/test.mp4'